Nama Kelompok :


1.   Ananda Ayu Sekar W
2.   M. Favian Apta S
3.   Ony Novianti
4.   Raihan Hidayatullah D
5.   Satria Alief Pratama S



In [ ]:
import  numpy  as  np 
import  pandas  as  pd 
import  re

import  nltk 
nltk.download ( 'wordnet' ) 
from  nltk.corpus  import  stopwords


#import  spasi

from  scipy.stats  import  itemfreq 
from  sklearn.model_selection  import  train_test_split 
from  sklearn.naive_bayes  import  MultinomialNB 
from  sklearn.pipeline  import  Pipeline 
from  sklearn.preprocessing  import  LabelEncoder 
from  sklearn.feature_extraction.text  import  CountVectorizer,TfidfTransformer,HashingVectorizer

pd.options.mode.chained_assignment =  None

[nltk_data] Downloading package wordnet to /root/nltk_data...


**Untuk membaca file tweet**

In [ ]:

df = pd.read_csv('tweet_emotions.csv')

df.head(10)

,tweet_id,sentiment,content
0,1956967341,empty,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,wants to hang out with friends SOON!
4,1956968416,neutral,@dannycastillo We want to trade with someone w...
5,1956968477,worry,Re-pinging @ghostridah14: why didn't you go to...
6,1956968487,sadness,"I should be sleep, but im not! thinking about ..."
7,1956968636,worry,Hmmm. http://www.djhero.com/ is down
8,1956969035,sadness,@charviray Charlene my love. I miss you
9,1956969172,sadness,@kelcouch I'm sorry at least it's Friday?


Melihat isi dari 

In [ ]:
df.shape

(40000, 3)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tweet_id   40000 non-null  int64 
 1   sentiment  40000 non-null  object
 2   content    40000 non-null  object
dtypes: int64(1), object(2)
memory usage: 937.6+ KB


In [ ]:
df = df[['tweet_id', 'sentiment', 'content']].copy()

In [ ]:
df.sentiment.value_counts()

neutral       8638
worry         8459
happiness     5209
sadness       5165
love          3842
surprise      2187
fun           1776
relief        1526
hate          1323
empty          827
enthusiasm     759
boredom        179
anger          110
Name: sentiment, dtype: int64

In [ ]:
df.sentiment = np.where((df.sentiment == 'neutral') | (df.sentiment == 'empty') | (df.sentiment == 'boredom'), 'neutral', df.sentiment)

In [ ]:
df.sentiment = np.where((df.sentiment == 'fun') | (df.sentiment == 'enthusiasm'), 'fun', df.sentiment)

In [ ]:
df = df[df.sentiment != 'neutral']

In [ ]:
df.sentiment.value_counts()

worry        8459
happiness    5209
sadness      5165
love         3842
fun          2535
surprise     2187
relief       1526
hate         1323
anger         110
Name: sentiment, dtype: int64

**Jika memakai tweet clean**

In [ ]:
# df2 = pd.read_csv('data/tweets_clean.txt',sep='	', header=None)

In [ ]:
# df2.head(10)

In [ ]:
# df2.columns=['tweet_id','content','sentiment']

In [ ]:
# df2.sentiment = df2.sentiment.str.replace(':: ','')

In [ ]:
# df2.sentiment.value_counts()

## **Emotions to keep**

worry, happpy (happiness,joy), surprise, sadness, love, fear, anger,hate (disgust+hate), relief, fun (fun+enthusiasm)

In [ ]:
# data = df.append(df2)
data = df
data.head(10)

,tweet_id,sentiment,content
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,Funeral ceremony...gloomy friday...
3,1956967789,fun,wants to hang out with friends SOON!
5,1956968477,worry,Re-pinging @ghostridah14: why didn't you go to...
6,1956968487,sadness,"I should be sleep, but im not! thinking about ..."
7,1956968636,worry,Hmmm. http://www.djhero.com/ is down
8,1956969035,sadness,@charviray Charlene my love. I miss you
9,1956969172,sadness,@kelcouch I'm sorry at least it's Friday?
11,1956969531,worry,Choked on her retainers
12,1956970047,sadness,Ugh! I have to beat this stupid song to get to...


In [ ]:
data.sentiment = np.where((data.sentiment == 'disgust') |(data.sentiment == 'hate'),'hate',data.sentiment)

In [ ]:
data=data[data.sentiment.isin(['sadness','worry','joy'])]

In [ ]:
data.sentiment.value_counts()

worry      8459
sadness    5165
Name: sentiment, dtype: int64

## **Membersihkan Teks**
##### Hapus karakter yang tidak relevan selain alfanumerik dan spasi

In [ ]:
data['content']=data['content'].str.replace('[^A-Za-z0-9\s]+', '')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


## **Hapus tautan dari teks**

In [ ]:
data['content']=data['content'].str.replace('http\S+|www.\S+', '', case=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


## **Ubah semuanya menjadi huruf kecil**

In [ ]:
data['content']=data['content'].str.lower()

## **Tetapkan Variabel Target**

In [ ]:
target=data.sentiment
data = data.drop(['sentiment'],axis=1)

In [ ]:
le=LabelEncoder()
target=le.fit_transform(target)

# **Split Data kedalam train & test**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data,target,stratify=target,test_size=0.4, random_state=42)

### Periksa apakah split membagi kelas secara seragam

In [ ]:
itemfreq(y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `itemfreq` is deprecated!
`itemfreq` is deprecated and will be removed in a future version. Use instead `np.unique(..., return_counts=True)`
  """Entry point for launching an IPython kernel.


array([[   0, 3099],
       [   1, 5075]])

In [ ]:
itemfreq(y_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `itemfreq` is deprecated!
`itemfreq` is deprecated and will be removed in a future version. Use instead `np.unique(..., return_counts=True)`
  """Entry point for launching an IPython kernel.


array([[   0, 2066],
       [   1, 3384]])

# **Tokenization**
#### Tokenization dapat dilakukan dengan berbagai cara, yaitu **Bag of words, tf-idf, Glove, word2vec ,fasttext** etc. Sekarang lakukan penerapannya dan pengaruhnya terhadap akurasi

##### **Bag of Words**

In [ ]:
# Extract fitur
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train.content)
X_test_counts =count_vect.transform(X_test.content)
print('Shape of Term Frequency Matrix: ',X_train_counts.shape)

Shape of Term Frequency Matrix:  (8174, 15811)


1. **Naive Bayess Model**

In [ ]:
# Training Naive Bayes (NB) menggunakan training data.
clf = MultinomialNB().fit(X_train_counts,y_train)
predicted = clf.predict(X_test_counts)
nb_clf_accuracy = np.mean(predicted == y_test) * 100
print(nb_clf_accuracy)

62.36697247706422


2. **Menggunakan Pipeline**
   * tentukan fungsi untuk akurasi pencetakan

In [ ]:

def print_acc(model):
    predicted = model.predict(X_test.content)
    accuracy = np.mean(predicted == y_test) * 100
    print(accuracy)

In [ ]:
nb_clf = Pipeline([('vect', CountVectorizer()), ('clf', MultinomialNB())])
nb_clf = nb_clf.fit(X_train.content,y_train)
print_acc(nb_clf)

62.36697247706422


3. **TF IDF transformer**

In [ ]:
# Training data menggunakan TF IDF Transformer
nb_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])
nb_clf = nb_clf.fit(X_train.content,y_train)
print_acc(nb_clf)

62.18348623853211


4. **Hash Vectorizer**
   * Naive Bayes membutuhkan input non-negatif. Oleh karena itu, tanda alternatif harus disetel ke false di Hashing Vectorizer untuk membuatnya bekerja dengan algoritma naive bayes

In [ ]:
# Training data dengan 
nb_clf = Pipeline([('vect', HashingVectorizer(n_features=2500,alternate_sign=False)), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])
nb_clf = nb_clf.fit(X_train.content,y_train)
print_acc(nb_clf)

61.72477064220183


In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,predicted)

array([[ 425, 1641],
       [ 410, 2974]])

5. **Hapus Stop Words**

In [ ]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
nb_clf = Pipeline([('vect', CountVectorizer(stop_words=stop_words)), ('clf', MultinomialNB())])
nb_clf = nb_clf.fit(X_train.content,y_train)
print_acc(nb_clf)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


62.605504587155956


In [ ]:
nb_clf = Pipeline([('vect', CountVectorizer(stop_words=stop_words)), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])
nb_clf = nb_clf.fit(X_train.content,y_train)
print_acc(nb_clf)

62.23853211009174


6. **Lemmatization**

In [ ]:
nltk.download('omw-1.4')
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return ' '.join([lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)])
X_train.loc[:,'content'] = X_train['content'].apply(lemmatize_text)
X_test.loc[:,'content'] = X_test['content'].apply(lemmatize_text)

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
nb_clf = Pipeline([('vect', CountVectorizer(stop_words=stop_words)), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])
nb_clf = nb_clf.fit(X_train.content,y_train)
print_acc(nb_clf)

62.36697247706422
